### NLP - Natural Language Processing (Procesamiento de Lenguaje Natural)

En este notebook se hará el procesamiento de lenguaje natural (NLP) en los comentarios de los datasets de entrenamiento y prueba. Se utilizará la librería `nltk` para realizar el procesamiento de los comentarios.

Lo que se hará es:
1. Convertir los comentarios a minúsculas
2. Tokenizar los comentarios
3. Eliminar contenido que hace ruido -> URLs, menciones, hashtags, etc.
4. Reducir palabras extendidas (como "Holaaaaaaa" a "Hola")
5. Eliminar las palabras vacías (stopwords)
6. limitar caracteres solo letras
7. Eliminar los espacios en blanco
8. Lematizar las palabras
9. Eliminar tokens que tenga menos de 3 caracteres

Luego una vez ya queden los tokens limpios, se entrenará un Word2Vec para obtener los vectores de palabras de los comentarios. (Promedio de los vectores de palabras de cada comentario)

In [1]:
from IPython.core.display import display, HTML; display(HTML('<style>.output_wrapper, .output {height:auto !important; max-height:150px;}.output_scroll {box-shadow:none !important; webkit-box-shadow:none !important;}</style>'))

/var/folders/kr/066v09l56c9_4l91brw4fwdc0000gn/T/ipykernel_26106/373857186.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML; display(HTML('<style>.output_wrapper, .output {height:auto !important; max-height:150px;}.output_scroll {box-shadow:none !important; webkit-box-shadow:none !important;}</style>'))


In [2]:
# !IGNORA ESTA CELDA!
# Esto es para que las celdas de la notebook se ajusten a la altura del contenido
from IPython.core.display import display, HTML; display(HTML('<style>.output_wrapper, .output {height:auto !important; max-height:150px;}.output_scroll {box-shadow:none !important; webkit-box-shadow:none !important;}</style>'))

/var/folders/kr/066v09l56c9_4l91brw4fwdc0000gn/T/ipykernel_26106/3703233119.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML; display(HTML('<style>.output_wrapper, .output {height:auto !important; max-height:150px;}.output_scroll {box-shadow:none !important; webkit-box-shadow:none !important;}</style>'))


In [3]:
# Instalación de librerías
%pip install -r '../requirements.txt'

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Importación de librerías
# Expresiones regulares
import re
# Manejo de datos
import pandas as pd
# Procesamiento de texto
import nltk
from nltk.corpus import stopwords
# Lematizador
import spacy
import os
import urllib.request
import json
from unidecode import unidecode

#### Instalación del modelo de Spacy en español para lematización

In [5]:
# Verificar si el modelo ya existe en la carpeta assets
if not os.path.exists('./assets/nlp_spacy/es_core_news_lg-3.1.0'):
  # Descargar el modelo
  url = 'https://github.com/explosion/spacy-models/releases/download/es_core_news_lg-3.1.0/es_core_news_lg-3.1.0.tar.gz'
  urllib.request.urlretrieve(url, './assets/nlp_spacy/es_core_news_lg.tar.gz')
  # Descomprimir el archivo
  os.system('tar -xf ./assets/nlp_spacy/es_core_news_lg.tar.gz -C ./assets/nlp_spacy')
  # Eliminar el archivo comprimido
  os.remove('assets/nlp_spacy/es_core_news_lg.tar.gz')

In [6]:
# Carga de datos
datos_entrenamiento = pd.read_csv('../2-traduccion-dataset/result/dataset-spanish-tweets-train-translated.csv', delimiter=',', encoding='utf-8')
datos_prueba = pd.read_csv('../2-traduccion-dataset/result/dataset-spanish-tweets-test-translated.csv', delimiter=',', encoding='utf-8')
datos_validacion = pd.read_csv('../2-traduccion-dataset/result/dataset-spanish-tweets-validation-translated.csv', delimiter=',', encoding='utf-8')

In [7]:
# Definición de clase para el preprocesamiento de datos

class Preprocesamiento:
  stopwords_es = []
  stopwords_en = []
  nlp = None

  def __init__(self):
    nltk.download('stopwords')
    nltk.download('punkt_tab')
    # Cargar el modelo de lenguaje en español
    self.nlp = spacy.load('assets/nlp_spacy/es_core_news_lg-3.1.0/es_core_news_lg/es_core_news_lg-3.1.0/')
    self.stopwords_es = stopwords.words('spanish')
    self.stopwords_en = stopwords.words('english')
    pass

  # Función principal para preprocesar texto
  def preprocesar_texto(self, texto: str) -> list[str]:
    nuevo_texto = self.pasar_a_minusculas(texto)
    tokens = self.tokenizar_texto(nuevo_texto)
    tokens = self.eliminar_ruido(tokens)
    tokens = self.reducir_palabras_extendidas(tokens)
    tokens = self.eliminar_stopwords(tokens)
    tokens = self.limitar_caracteres(tokens)
    tokens = self.eliminar_espacios(tokens)
    tokens = self.lematizar_tokens(tokens)
    tokens = self.eliminar_tokens_cortos(tokens)
    return tokens

  # Paso 1: Convertir el texto a minúsculas
  def pasar_a_minusculas(self, texto: str) -> str:
    texto_str = str(texto)
    return texto_str.lower()

  # Paso 2: Tokenizar el texto
  # Ejemplo: "Hola, ¿cómo estás?" -> ["Hola,", "¿cómo", "estás?"]
  def tokenizar_texto(self, texto: str) -> list[str]:
    texto_tokenizado = texto.split(' ')
    texto_tokenizado = self.limpiar_tokens(texto_tokenizado)
    texto_tokenizado_limpio = self.eliminar_ruido(texto_tokenizado)
    texto_tokenizado_limpio = self.reducir_palabras_extendidas(texto_tokenizado_limpio)
    return texto_tokenizado_limpio

  # Paso 3: Eliminar ruido como URLs, menciones y hashtags
  # Ejemplo: ["Hola,"@sebastian, "http://www.google.com", "#InteligenciaArtificial", "¿cómo", "estás?"] -> ["Hola,", "¿cómo", "estás?"]
  def eliminar_ruido(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    # Eliminar URLs
    lista_limpia = []
    for token in tokens:
      if not token.startswith('http'):
        lista_limpia.append(token)
    # Eliminar Hashtags
    lista_limpia = [token for token in lista_limpia if not token.startswith('#')]
    # Eliminar menciones
    lista_limpia = [token for token in lista_limpia if not token.startswith('@')]
    return lista_limpia

  # Paso 4: Reducir palabras extendidas
  # Ejemplo: ["Holaaaa"] -> ["Hola"]
  def reducir_palabras_extendidas(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    for i in range(len(tokens)):
      tokens[i] = re.sub(r'(.)\1{2,}', r'\1\1', tokens[i])
    return tokens

  # Paso 5: Eliminar stopwords
  def eliminar_stopwords(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    tokens_limpio = [token for token in tokens if token not in self.stopwords_es]
    tokens_limpio = [token for token in tokens_limpio if token not in self.stopwords_en]
    return tokens_limpio

  # Paso 6: Limitar caracteres solo a letras
  def limitar_caracteres(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    tokens_limpio = self.eliminar_acentos(tokens)
    tokens_limpio = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens_limpio]
    tokens_limpio = self.limpiar_tokens(tokens_limpio)
    return tokens_limpio

  # Paso 7: Eliminar espacios en blanco adicionales
  def eliminar_espacios(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    tokens_limpio = self.limpiar_tokens(tokens)
    tokens_limpio = [token.strip() for token in tokens]
    return tokens_limpio

  # Paso 8: Lematizar tokens
  def lematizar_tokens(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    tokens_lematizados = []
    for token in tokens:
        doc = self.nlp(token)
        # Obtener la primera palabra lematizada
        lemma = doc[0].lemma_.split()[0]
        tokens_lematizados.append(lemma)
    tokens_lematizados = self.eliminar_acentos(tokens_lematizados)
    return tokens_lematizados

  # Paso 9: Eliminar tokens con longitud menor a 3
  def eliminar_tokens_cortos(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    tokens_limpio = [token for token in tokens if len(token) > 2]
    return tokens_limpio

  # Helper 1: Limpiar tokens
  # Ejemplo: ["Hola,", "", "¿cómo", "estás?"] -> ["Hola,", "¿cómo", "estás?"]
  def limpiar_tokens(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    tokens_limpio = [token for token in tokens if token != '']
    return tokens_limpio

  # Helper 2: Eliminar acentos
  # Ejemplo: ["Hóla", "¿cómo", "estás?"] -> ["Hola", "¿como", "estas?"]
  def eliminar_acentos(self, tokens: list[str]) -> list[str]:
    if len(tokens) == 0: return []
    tokens_copy = tokens.copy()

    for i in range(len(tokens_copy)):
      tokens_copy[i] = unidecode(tokens_copy[i])
      tokens_copy[i] = tokens_copy[i].replace('ñ', 'n')
    return tokens_copy

In [8]:
# Preprocesamiento de datos
preprocesamiento = Preprocesamiento()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sebastiancb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sebastiancb/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/Users/sebastiancb/anaconda3/envs/env-talentotech-project/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'es_core_news_lg' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [9]:
# Aplicar preprocesamiento a los datos de entrenamiento
datos_entrenamiento['text'] = datos_entrenamiento['text'].apply(lambda x: preprocesamiento.preprocesar_texto(x))
datos_prueba['text'] = datos_prueba['text'].apply(lambda x: preprocesamiento.preprocesar_texto(x))
datos_validacion['text'] = datos_validacion['text'].apply(lambda x: preprocesamiento.preprocesar_texto(x))

In [10]:
# Remover filas con textos donde la longitud sea 0
datos_entrenamiento = datos_entrenamiento[datos_entrenamiento['text'].map(len) > 0]
datos_prueba = datos_prueba[datos_prueba['text'].map(len) > 0]
datos_validacion = datos_validacion[datos_validacion['text'].map(len) > 0]

In [11]:
# Convertir JSON la columna 'text' para guardar en CSV
datos_entrenamiento['text'] = datos_entrenamiento['text'].apply(lambda x: json.dumps(x))
datos_prueba['text'] = datos_prueba['text'].apply(lambda x: json.dumps(x))
datos_validacion['text'] = datos_validacion['text'].apply(lambda x: json.dumps(x))

In [12]:
# Guardar los datos preprocesados
datos_entrenamiento.to_csv('./dist/dataset-spanish-tweets-train-preprocessed.csv', index=False, encoding='utf-8')
datos_prueba.to_csv('./dist/dataset-spanish-tweets-test-preprocessed.csv', index=False, encoding='utf-8')
datos_validacion.to_csv('./dist/dataset-spanish-tweets-validation-preprocessed.csv', index=False, encoding='utf-8')